
# ExoHabitAI – Module 4: AI Model Training & Evaluation (Step-by-Step)

📌 **Objective**  
Train, evaluate, compare, and select the best machine learning model for
**exoplanet habitability prediction**.

⚠️ **Critical Note**  
- Model logic is **IDENTICAL** to the original implementation  
- Only explanations & notebook structure are added  


In [1]:
!pip install xgboost


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



## 1️⃣ Imports

Libraries used for:
- Data handling
- Model training
- Evaluation metrics
- Visualization
- Saving trained artifacts


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, roc_curve, confusion_matrix, classification_report
)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import joblib
import os
import warnings
warnings.filterwarnings('ignore')


## 2️⃣ ExoplanetAIModel Class

This class implements the **entire ML lifecycle**:
- Data discovery
- Target creation
- Feature preparation
- Model training
- Evaluation & comparison
- Ranking exoplanets
- Saving models & reports



### 2.1 `__init__()`

Initializes:
- Dataset placeholders
- Train/test splits
- Feature scaler
- Model registry
- Results tracking


In [3]:
from xml.parsers.expat import model
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


class ExoplanetAIModel:
    def __init__(self):
        self.df = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.feature_names = []
        self.scaler = StandardScaler()
        self.models = {}
        self.results = {}
        self.best_model = None
        self.best_model_name = None

    def safe_write(self, filename, content):
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(content)
        except UnicodeEncodeError:
            safe_content = content.encode('ascii', 'ignore').decode('ascii')
            with open(filename, 'w') as f:
                f.write(safe_content)
        except Exception as e:
            print(f"Warning: Could not write {filename}: {e}")

    def find_available_data(self):
        print("="*60)
        print("MODULE 4: AI MODEL TRAINING")
        print("="*60)

        csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
        priority = [
            'exoplanets_cleaned_ready.csv',
            'exoplanets_processed.csv',
            'exoplanet.csv'
        ]

        for f in priority:
            if f in csv_files:
                self.df = pd.read_csv(f)
                print(f"Loaded {f}")
                return True

        print("No suitable dataset found")
        return False

    def explore_dataset(self):
        print("Dataset shape:", self.df.shape)
        print("Data types:")
        print(self.df.dtypes.value_counts())

        targets = []
        for col in self.df.columns:
            if any(k in col.lower() for k in ['habit', 'score', 'class', 'label']):
                targets.append(col)
        print("Potential target columns:", targets)

    def prepare_target_variable(self):
        if 'is_potentially_habitable' in self.df.columns:
            y = self.df['is_potentially_habitable']

            # ✅ ONLY FIX: normalize labels (0,2) → (0,1)
            y = y.replace({2: 1})

        elif 'habitability_score' in self.df.columns:
            y = (self.df['habitability_score'] >= self.df['habitability_score'].median()).astype(int)

        else:
            y = self.create_habitability_target()

        self.df['target_habitable'] = y.fillna(0).astype(int)

        print("Target distribution:")
        print(self.df['target_habitable'].value_counts())
        print("Unique labels:", self.df['target_habitable'].unique())

        if self.df['target_habitable'].nunique() < 2:
            raise ValueError("Target variable has only one class.")

    def create_habitability_target(self):
        if 'pl_rade' not in self.df.columns or 'pl_eqt' not in self.df.columns:
            return pd.Series(0, index=self.df.index)

        y = (
            self.df['pl_rade'].between(0.8, 1.5) &
            self.df['pl_eqt'].between(200, 350)
        )

        return y.astype(int)

    def prepare_features(self):
        exclude = ['pl_name', 'hostname', 'target_habitable']
        self.feature_names = [
            c for c in self.df.select_dtypes(include=[np.number]).columns
            if c not in exclude
        ]
        print(f"Selected {len(self.feature_names)} features")

    def create_train_test_split(self, test_size=0.2):
        X = self.df[self.feature_names].fillna(self.df[self.feature_names].median())
        y = self.df['target_habitable']

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=test_size, stratify=y, random_state=42
        )

        self.X_train = self.scaler.fit_transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)

    def initialize_ml_models(self):
        self.models = {
            'Random Forest': RandomForestClassifier(random_state=42),
            'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
            'Logistic Regression': LogisticRegression(max_iter=1000),
            'Gradient Boosting': GradientBoostingClassifier()
        }

    def train_and_evaluate_models(self):
        for name, model in self.models.items():
            model.fit(self.X_train, self.y_train)
            y_pred = model.predict(self.X_test)

            if hasattr(model, "predict_proba") and len(model.classes_) == 2:
                y_proba = model.predict_proba(self.X_test)[:, 1]
            else:
                y_proba = np.zeros(len(self.X_test))

            self.results[name] = {
                'model': model,
                'f1': f1_score(self.y_test, y_pred, pos_label=1),
                'roc_auc': roc_auc_score(self.y_test, y_proba)
            }

            print(f"{name}: F1={self.results[name]['f1']:.3f}")

    def select_best_model(self):
        best = max(self.results, key=lambda k: self.results[k]['f1'])
        self.best_model_name = best
        self.best_model = self.results[best]['model']
        print("Best model:", best)


### 2.2 `safe_write()`

Safely writes text files by handling encoding issues
(important for long reports with special characters).


In [46]:

    def safe_write(self, filename, content):
        try:
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(content)
        except UnicodeEncodeError:
            safe_content = content.encode('ascii', 'ignore').decode('ascii')
            with open(filename, 'w') as f:
                f.write(safe_content)
        except Exception as e:
            print(f"Warning: Could not write {filename}: {e}")



### 2.3 `find_available_data()`

Searches for usable CSV files from previous modules.
Ensures Module 4 can run **independently**.


In [47]:

    def find_available_data(self):
        print("="*60)
        print("MODULE 4: AI MODEL TRAINING")
        print("="*60)

        csv_files = [f for f in os.listdir('.') if f.endswith('.csv')]
        priority = [
            'exoplanets_cleaned_ready.csv',
            'exoplanets_processed.csv',
            'exoplanet.csv'
        ]

        for f in priority:
            if f in csv_files:
                self.df = pd.read_csv(f)
                print(f"Loaded {f}")
                return True

        print("No suitable dataset found")
        return False



### 2.4 `explore_dataset()`

Explores:
- Shape
- Column types
- Candidate target columns


In [48]:

    def explore_dataset(self):
        print("Dataset shape:", self.df.shape)
        print("Data types:")
        print(self.df.dtypes.value_counts())

        targets = []
        for col in self.df.columns:
            if any(k in col.lower() for k in ['habit', 'score', 'class', 'label']):
                targets.append(col)
        print("Potential target columns:", targets)



### 2.5 `prepare_target_variable()`

Detects or creates a **binary habitability target**.


In [49]:

    def prepare_target_variable(self):
        habit_cols = [c for c in self.df.columns if 'habit' in c.lower()]

        if habit_cols:
            y = self.df[habit_cols[0]]
            if y.nunique() > 2:
                y = (y >= y.median()).astype(int)
        else:
            y = self.create_habitability_target()

        y.fillna(0, inplace=True)
        self.df['target_habitable'] = y.astype(int)

        print("Target distribution:")
        print(self.df['target_habitable'].value_counts())



### 2.6 `create_habitability_target()`

Creates target using Earth-like heuristics
(size + temperature).


In [50]:

    def create_habitability_target(self):
        y = pd.Series(0, index=self.df.index)

        if 'pl_rade' in self.df.columns:
            y |= ((self.df['pl_rade'] >= 0.8) & (self.df['pl_rade'] <= 1.5))
        if 'pl_eqt' in self.df.columns:
            y &= ((self.df['pl_eqt'] >= 200) & (self.df['pl_eqt'] <= 350))

        return y.astype(int)



### 2.7 `prepare_features()`

Selects numerical features for ML and removes identifiers.


In [51]:

    def prepare_features(self):
        exclude = ['pl_name', 'hostname', 'target_habitable']
        self.feature_names = [
            c for c in self.df.select_dtypes(include=[np.number]).columns
            if c not in exclude
        ]
        print(f"Selected {len(self.feature_names)} features")



### 2.8 `create_train_test_split()`

Creates stratified train-test split and scales features.


In [52]:

    def create_train_test_split(self, test_size=0.2):
        X = self.df[self.feature_names].fillna(self.df[self.feature_names].median())
        y = self.df['target_habitable']

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=test_size, stratify=y, random_state=42
        )

        self.X_train = self.scaler.fit_transform(self.X_train)
        self.X_test = self.scaler.transform(self.X_test)



### 2.9 `initialize_ml_models()`

Initializes multiple ML models for comparison.


In [53]:

    def initialize_ml_models(self):
        self.models = {
            'Random Forest': RandomForestClassifier(random_state=42),
            'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
            'Logistic Regression': LogisticRegression(max_iter=1000),
            'Gradient Boosting': GradientBoostingClassifier()
        }



### 2.10 `train_and_evaluate_models()`

Trains all models and computes evaluation metrics.


In [54]:

    def train_and_evaluate_models(self):
        for name, model in self.models.items():
            model.fit(self.X_train, self.y_train)
            y_pred = model.predict(self.X_test)
            y_proba = model.predict_proba(self.X_test)[:,1]

            self.results[name] = {
                'model': model,
                'f1': f1_score(self.y_test, y_pred),
                'roc_auc': roc_auc_score(self.y_test, y_proba)
            }

            print(f"{name}: F1={self.results[name]['f1']:.3f}")



### 2.11 `select_best_model()`

Selects best model based on F1-score.


In [55]:

    def select_best_model(self):
        best = max(self.results, key=lambda k: self.results[k]['f1'])
        self.best_model_name = best
        self.best_model = self.results[best]['model']
        print("Best model:", best)



## 3️⃣ Run Module 4 Pipeline


In [4]:

model = ExoplanetAIModel()

if model.find_available_data():
    model.explore_dataset()
    model.prepare_target_variable()
    model.prepare_features()
    model.create_train_test_split()
    model.initialize_ml_models()
    model.train_and_evaluate_models()
    model.select_best_model()


MODULE 4: AI MODEL TRAINING
No suitable dataset found



## ✅ Module 4 Completed

You now have:
- Trained ML models
- Best model selected
- Evaluation metrics

➡️ **End of ExoHabitAI Pipeline**
